In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df_ml = pd.read_csv("../data/data_ml.csv")

In [ ]:
separation_date = "2013-12-31"
df_train = df_ml.query("date < @separation_date").copy()
df_test = df_ml.query("@separation_date <= date & date < '2018-12-31'").copy()

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
columns_to_drop = [
    "stock_id", "date", # non-feature identifiers
    "R1M_Usd", "R3M_Usd", "R6M_Usd", "R12M_Usd", # numerical labels
    "R1M_Usd_C", "R12M_Usd_C" # categorical labels
]

features = list(df_ml.drop(columns=columns_to_drop).columns)

In [ ]:
X_train = df_train[features]
y_train = df_train[["R1M_Usd_C"]]

In [ ]:
X_test = df_test[features]
y_test = df_test[["R1M_Usd_C"]]

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression()

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cross_val_score(model, X_train, y_train.values.ravel())

array([0.51434416, 0.52189505, 0.52598616, 0.51686954, 0.51346028])

In [ ]:
model.fit(X_train, y_train.values.ravel())

LogisticRegression()

In [ ]:
df_test["prediction_hard"] = model.predict(X_test)
df_test["predict_prob_0"] = model.predict_proba(X_test)[:,0]
df_test["predict_prob_1"] = model.predict_proba(X_test)[:,1]

In [ ]:
df_screen = df_test[["date", "R1M_Usd", "prediction_hard", "predict_prob_0", "predict_prob_1"]]
df_screen

,date,R1M_Usd,prediction_hard,predict_prob_0,predict_prob_1
3,2015-03-31,0.174,1.0,0.470104,0.529896
4,2015-04-30,-0.106,1.0,0.470435,0.529565
5,2015-05-31,-0.185,1.0,0.470004,0.529996
6,2015-06-30,-0.418,1.0,0.459854,0.540146
7,2015-07-31,0.575,1.0,0.457712,0.542288
...,...,...,...,...,...
269373,2018-07-31,0.028,0.0,0.541795,0.458205
269374,2018-08-31,-0.101,0.0,0.540781,0.459219
269375,2018-09-30,0.013,0.0,0.508696,0.491304
269376,2018-10-31,0.039,0.0,0.506801,0.493199


In [ ]:
rebalance_dates = list(np.sort(df_screen["date"].unique()))
rebalance_dates

['2013-12-31',
 '2014-01-31',
 '2014-02-28',
 '2014-03-31',
 '2014-04-30',
 '2014-05-31',
 '2014-06-30',
 '2014-07-31',
 '2014-08-31',
 '2014-09-30',
 '2014-10-31',
 '2014-11-30',
 '2014-12-31',
 '2015-01-31',
 '2015-02-28',
 '2015-03-31',
 '2015-04-30',
 '2015-05-31',
 '2015-06-30',
 '2015-07-31',
 '2015-08-31',
 '2015-09-30',
 '2015-10-31',
 '2015-11-30',
 '2015-12-31',
 '2016-01-31',
 '2016-02-29',
 '2016-03-31',
 '2016-04-30',
 '2016-05-31',
 '2016-06-30',
 '2016-07-31',
 '2016-08-31',
 '2016-09-30',
 '2016-10-31',
 '2016-11-30',
 '2016-12-31',
 '2017-01-31',
 '2017-02-28',
 '2017-03-31',
 '2017-04-30',
 '2017-05-31',
 '2017-06-30',
 '2017-07-31',
 '2017-08-31',
 '2017-09-30',
 '2017-10-31',
 '2017-11-30',
 '2017-12-31',
 '2018-01-31',
 '2018-02-28',
 '2018-03-31',
 '2018-04-30',
 '2018-05-31',
 '2018-06-30',
 '2018-07-31',
 '2018-08-31',
 '2018-09-30',
 '2018-10-31',
 '2018-11-30']

In [ ]:
df_screen.query("date == '2013-12-31'").sort_values(by="predict_prob_1", ascending=False).head(100) 

,date,R1M_Usd,prediction_hard,predict_prob_0,predict_prob_1
5107,2013-12-31,-0.055,1.0,0.394118,0.605882
109925,2013-12-31,0.114,1.0,0.394824,0.605176
246437,2013-12-31,0.080,1.0,0.397836,0.602164
132222,2013-12-31,-0.035,1.0,0.411254,0.588746
195949,2013-12-31,0.033,1.0,0.412173,0.587827
...,...,...,...,...,...
146911,2013-12-31,-0.054,1.0,0.462449,0.537551
81913,2013-12-31,0.007,1.0,0.462745,0.537255
194089,2013-12-31,0.006,1.0,0.462818,0.537182
198657,2013-12-31,0.038,1.0,0.463075,0.536925


In [ ]:
lst_trades = []
for ix_date in rebalance_dates:
    #print(ix_date)
    df = df_screen.query("date == @ix_date").sort_values(by="predict_prob_1", ascending=False).head(200).copy()
    lst_trades.append(df)
df_trades = pd.concat(lst_trades)
df_trades

,date,R1M_Usd,prediction_hard,predict_prob_0,predict_prob_1
5107,2013-12-31,-0.055,1.0,0.394118,0.605882
109925,2013-12-31,0.114,1.0,0.394824,0.605176
246437,2013-12-31,0.080,1.0,0.397836,0.602164
132222,2013-12-31,-0.035,1.0,0.411254,0.588746
195949,2013-12-31,0.033,1.0,0.412173,0.587827
...,...,...,...,...,...
244159,2018-11-30,-0.206,1.0,0.472301,0.527699
45904,2018-11-30,-0.155,1.0,0.472338,0.527662
124747,2018-11-30,-0.313,1.0,0.472515,0.527485
120197,2018-11-30,-0.100,1.0,0.472560,0.527440


In [ ]:
df_monthly_returns = df_trades.groupby("date")[["R1M_Usd"]].mean().reset_index()
df_monthly_returns['growth_factor'] = 1 + df_monthly_returns['R1M_Usd']
df_monthly_returns['equity_curve'] = df_monthly_returns['growth_factor'].cumprod()
df_monthly_returns

,date,R1M_Usd,growth_factor,equity_curve
0,2013-12-31,-0.029485,0.970515,0.970515
1,2014-01-31,0.054355,1.054355,1.023267
2,2014-02-28,0.021750,1.021750,1.045523
3,2014-03-31,0.001840,1.001840,1.047447
4,2014-04-30,0.005995,1.005995,1.053727
5,2014-05-31,0.036100,1.036100,1.091766
6,2014-06-30,-0.052200,0.947800,1.034776
7,2014-07-31,0.050005,1.050005,1.086520
8,2014-08-31,-0.060395,0.939605,1.020900
9,2014-09-30,0.074480,1.074480,1.096936
